Here, we perform a deep mutational scan on an enzyme-TSS complex. We are separating this step from comparative modeling steps purely for convenience in scheduling on Cabernet (also the CM right now is not all in Python/PyRosetta). This notebook assumes the existance of the output directory of the comparative modeling protocol, with all the stuff in it (_i.e._ the CM pipeline as been run). 

The output will be a HDF file with a tidy DataFrame (or multiple H5 files containing such as residue level features, and another containing enzyme design features. Actually, enzyme design features are strictly a subset of the residue energy features, provided you know the residue ID each catalytic residue and the ligand. 

Of course, we can programmatically obtain these values. So it seems like what we want to a) use the features reporter to obtain residue energies, or use a Python loop to write to Pandas, which can write out our flat files. 

In [1]:
import os 
import sys 
from random import choice
from itertools import product 

import pandas 
import pyrosetta 
from rosetta import core, protocols
from Bio.Data import IUPACData

In [2]:
# conveinence function for doing all output in parallel 
# get environment for parallel, if we want to do that 

task_id = os.getenv('SLURM_ARRAY_TASK_ID') 
task_id = '2' # for debug!
task_index = int(task_id) - 1 # damn it 
output_dir = 'example_input'
to_do_list = []
from glob import glob 
dirs = glob(os.path.join(output_dir, '*'))
for directory in dirs:
    to_do_list.append(directory) 

print('Performing "{}", task {}'.format(to_do_list[task_index], task_index))
os.chdir(to_do_list[task_index])
print(glob('*fa'))

Performing "example_input/100", task 1
['freezer_targets.fa', 'known.fa', 'targets.fa']


In [3]:
pyrosetta.init('-preserve_header -beta -extra_res_fa pnpg.params')
pose = pyrosetta.pose_from_file('native.pdb')
beta = pyrosetta.create_score_function('beta_cst')

emo = core.scoring.methods.EnergyMethodOptions()
emo.hbond_options().decompose_bb_hb_into_pair_energies(True)
beta.set_energy_method_options( emo )

beta(pose)

Found rosetta database at: /Users/alex/anaconda3/envs/rose/lib/python3.5/site-packages/pyrosetta-4.0-py3.5.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:94998f0e5b038b6d8083995a664cb96641cb844b 2017-05-11 15:10:16 -0500] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



-1421.8567659243722

In [4]:
# add in match constraints (could also use regular restraints) 
add_cst = protocols.enzdes.AddOrRemoveMatchCsts()
add_cst.set_cst_action(protocols.enzdes.ADD_NEW)
add_cst.cstfile('pnpg.cst')
add_cst.apply(pose)

beta(pose)

-1436.3334875820815

At this point, the pose is relaxed, docked, and ready to go. This represents, we think, a physically realistic model of the enzyme-TSS complex. For this particular family, we have based the geometry on a 2.7 Å crystal structure (Isorna 2007) with a bound transition state inhibitor (PDB 2jie).   

We need two lists to iterate over (residues we wish to mutate, and residues we wish to mutate to at each position. 

In [5]:
score_types = beta.get_nonzero_weighted_scoretypes()

In [ ]:
def deep_mutational_scan():
    for residue_index in range(1, pose.total_residue()+1):
        index = core.select.residue_selector.ResidueIndexSelector(str(residue_index))
        hood = core.select.residue_selector.NeighborhoodResidueSelector(index, 10.0)
        print('hood applied to pose\n', hood.apply(pose), 'position', residue_index)
        for three_letter, one_letter in IUPACData.protein_letters_3to1.items():
            copy_pose = pose.clone()
            mutant_name = '{}{}{}'.format(pose.sequence()[residue_index-1], residue_index, one_letter)
            print(mutant_name)
            mutate = protocols.simple_moves.MutateResidue(residue_index, three_letter.upper()) 
            mutate.apply(copy_pose) 
            print('After mutate:', beta(copy_pose))
            tf = core.pack.task.TaskFactory()
            pt = tf.create_packer_task(copy_pose)
            pt.restrict_to_repacking()
            pt.restrict_to_residues(hood.apply(copy_pose))
            #minpack = protocols.simple_moves.MinPackMover(beta, pt) 
            #minpack.apply(copy_pose)
            print('After min_pack:', beta(copy_pose))
            yield pandas.Series({'{}__{}'.format(i, core.scoring.name_from_score_type(st)): copy_pose.energies().residue_total_energies(residue_index)[st] for i, st in product(range(1, pose.total_residue()+1), score_types)}, name=mutant_name)

In [ ]:
data = deep_mutational_scan()
df = pandas.DataFrame(data) 

hood applied to pose
 vector1_bool[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

After min_pack: -1408.1941848166366
E3V
After mutate: -1436.6243776646697
After min_pack: -1436.6243776646697
hood applied to pose
 vector1_bool[1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

After mutate: -808.9647676506947
After min_pack: -808.9647676506947
K6V
After mutate: -1402.6661860807537
After min_pack: -1402.6661860807537
hood applied to pose
 vector1_bool[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

After mutate: -1439.2988922421496
After min_pack: -1439.2988922421496
T9W
After mutate: 980.7705834280258
After min_pack: 980.7705834280258
T9V
After mutate: -1331.2734363238549
After min_pack: -1331.2734363238549
hood applied to pose
 vector1_bool[0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

N12S
After mutate: -1426.4208623232166
After min_pack: -1426.4208623232166
N12A
After mutate: -1434.5081778327315
After min_pack: -1434.5081778327315
N12W
After mutate: -913.6760361648098
After min_pack: -913.6760361648098
N12V
After mutate: -1407.8016102070592
After min_pack: -1407.8016102070592
hood applied to pose
 vector1_bool[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

...

T122W
After mutate: 126.35648856344683
After min_pack: 126.35648856344683
T122V
After mutate: -1428.327481840411
After min_pack: -1428.327481840411
hood applied to pose
 vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

P125S
After mutate: -1423.4063488852955
After min_pack: -1423.4063488852955
P125A
After mutate: -1435.8724757451073
After min_pack: -1435.8724757451073
P125W
After mutate: 511.44692729859935
After min_pack: 511.44692729859935
P125V
After mutate: -1388.9876195334284
After min_pack: -1388.9876195334284
hood applied to pose
 vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

After min_pack: -1085.9919409604402
S129G
After mutate: -1434.9303306734264
After min_pack: -1434.9303306734264
S129F
After mutate: -110.18618804399375
After min_pack: -110.18618804399375
S129K
After mutate: -463.76264479348544
After min_pack: -463.76264479348544
S129P
After mutate: 1593.1091475983815
After min_pack: 1593.1091475983815
S129M
After mutate: -453.3484999081947
After min_pack: -453.3484999081947
S129T
After mutate: -1330.6399670746785
After min_pack: -1330.6399670746785
S129R
After mutate: 299.758791049788
After min_pack: 299.758791049788
S129D
After mutate: -1377.1173821872294
After min_pack: -1377.1173821872294
S129Y
After mutate: 200.50151817319733
After min_pack: 200.50151817319733
S129I
After mutate: -1251.6061833196575
After min_pack: -1251.6061833196575
S129E
After mutate: -1171.0139104240136
After min_pack: -1171.0139104240136
S129L
After mutate: -1250.0993291174204
After min_pack: -1250.0993291174204
S129N
After mutate: -1212.964538699897
After min_pack: -1212.964

After min_pack: -997.3978692544554
V132G
After mutate: -1431.8559012441658
After min_pack: -1431.8559012441658
V132F
After mutate: -382.792177527671
After min_pack: -382.792177527671
V132K
After mutate: -781.6506816930884
After min_pack: -781.6506816930884
V132P
After mutate: -230.7052455382364
After min_pack: -230.7052455382364
V132M
After mutate: -741.4126462211484
After min_pack: -741.4126462211484
V132T
After mutate: -1300.1574055064164
After min_pack: -1300.1574055064164
V132R
After mutate: -700.2452173777649
After min_pack: -700.2452173777649
V132D
After mutate: -1414.5633196393337
After min_pack: -1414.5633196393337
V132Y
After mutate: 69.98021855674807
After min_pack: 69.98021855674807
V132I
After mutate: -1036.6533026610712
After min_pack: -1036.6533026610712
V132E
After mutate: -1068.920076808037
After min_pack: -1068.920076808037
V132L
After mutate: -1206.0993548835022
After min_pack: -1206.0993548835022
V132N
After mutate: -1402.7273778278573
After min_pack: -1402.727377827

A135G
After mutate: -1435.898089731062
After min_pack: -1435.898089731062
A135F
After mutate: -964.9637445056018
After min_pack: -964.9637445056018
A135K
After mutate: -528.5428305609767
After min_pack: -528.5428305609767
A135P
After mutate: -955.4978736309472
After min_pack: -955.4978736309472
A135M
After mutate: -305.57905696187845
After min_pack: -305.57905696187845
A135T
After mutate: -1414.53903665287
After min_pack: -1414.53903665287
A135R
After mutate: -369.35139364239353
After min_pack: -369.35139364239353
A135D
After mutate: -1348.6229486712598
After min_pack: -1348.6229486712598
A135Y
After mutate: -956.5698687879218
After min_pack: -956.5698687879218
A135I
After mutate: -912.4338827538418
After min_pack: -912.4338827538418
A135E
After mutate: -758.4427812385675
After min_pack: -758.4427812385675
A135L
After mutate: -961.805346170171
After min_pack: -961.805346170171
A135N
After mutate: -1344.8799963987206
After min_pack: -1344.8799963987206
A135H
After mutate: -1111.42739566

In [ ]:
df.to_

In [ ]:
def get_residues_with_constraints(pose):    
    cst_set = pose.constraint_set()
    for n in cst_set.get_all_constraints():
        for residue in n.residues():
            yield int(residue)
            
set(get_residues_with_constraints(pose))

In [ ]:
def get_enzyme_design_features(pose, score_function):
    residues_with_constraints = set(get_residues_with_constraints(pose))
    energies = pose.energies()
    score_types = score_function.get_nonzero_weighted_scoretypes()

    for n, r in enumerate(residues_with_constraints, 1):
        for st in score_types:
            val = energies.residue_total_energies(r)[st]
            st_str = str(st).replace('ScoreType.', '')
            yield 'SR_{}_{}'.format(n, st_str), val

In [ ]:
# def deep_mutational_scan(pose, score_function, mask=None):
#     '''
    
#     deep_mutational_scan(pose: Pose, score_function: ScoreFunction, mask: int or boolean mask) 
#     Returns: 
    
#         pandas.DataFrame  
#     ''' 
    
#     targets = []
#     for n, native in enumerate(pose.sequence(), 1):
#         if native != 'Z':
#             for one_letter_code, three_letter_code in IUPACData.protein_letters_1to3.items():
#                 pkg = '{}{}{}'.format(native, n, one_letter_code)
#                 targets.append(pkg)
#     print(len(targets))

#     if isinstance(mask, int):
#         print("Applying mask")
#         # mask is an int, that means we only want the first `mask` 
#         # samples from the deep mutational scan (for debug) 
#         targets = [choice(targets) for n in range(mask)]
#         print(targets)
#     #elif len(mask) == pose.total_residue():
#         # this is a boolean mask 
#     #    targets = [t for m, t in zip(mask, targets) if m == 1]          
    
#     for n, native in enumerate(pose.sequence(), 1):
#         copy_pose = pose.clone()
#         for one_letter_code, three_letter_code in IUPACData.protein_letters_1to3.items():     
#             name = '{}{}{}'.format(native, n, one_letter_code)
#             if name in targets: 
#                 count += 1 
                
#                 try:
                    
#                     mutate = protocols.simple_moves.MutateResidue(n, three_letter_code.upper())
#                     mutate.apply(pose)
#                     total_score = score_function(copy_pose) 
#                     features = pandas.Series(dict(get_enzyme_design_features(copy_pose, score_function)))
#                     features.name = name

#                     #sys.stdout.write("\rFinished " + features.name)
#                     #sys.stdout.flush()

#                     yield features 
#                 except:
#                     print("Failed on", name)
            
# df = pandas.DataFrame(deep_mutational_scan(pose, beta)) 
# df.head()

# df.to_csv('dms_features.csv') 